In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import pickle

from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
import warnings
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


# 0 Importing Data

In [2]:
# Load your dataset
merge_df_scaled = pd.read_csv('../raw_data/another_try_top10.csv')
merge_df_scaled['date'] = pd.to_datetime(merge_df_scaled['date'])
merge_df_scaled.set_index('date', inplace=True)

merge_df_scaled
# 382600 rows × 64 columns

,id,item_id,dept_id,store_id,state_id,sales,weekday,wday,month,year,...,rolling_sum_120,rolling_mean_7,rolling_mean_30,rolling_mean_60,rolling_mean_90,rolling_stdv_7,rolling_stdv_30,rolling_stdv_60,rolling_stdv_90,rolling_stdv_120
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,FOODS_2_197_CA_1_validation,FOODS_2_197,FOODS_2,CA_1,CA,38,Saturday,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_3_080_CA_1_validation,FOODS_3_080,FOODS_3,CA_1,CA,33,Saturday,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_3_090_CA_1_validation,FOODS_3_090,FOODS_3,CA_1,CA,107,Saturday,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_3_120_CA_1_validation,FOODS_3_120,FOODS_3,CA_1,CA,0,Saturday,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-29,FOODS_3_252_CA_1_validation,FOODS_3_252,FOODS_3,CA_1,CA,19,Saturday,1,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-24,FOODS_3_555_CA_1_validation,FOODS_3_555,FOODS_3,CA_1,CA,24,Sunday,2,4,1.0,...,3169.0,33.571429,30.633333,27.083333,28.555556,19.789367,23.180675,19.909364,20.442911,19.716263
2016-04-24,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,CA_1,CA,54,Sunday,2,4,1.0,...,3194.0,41.285714,30.933333,27.483333,28.800000,14.279522,23.429915,20.207812,20.615365,19.875319
2016-04-24,FOODS_3_587_CA_1_validation,FOODS_3_587,FOODS_3,CA_1,CA,26,Sunday,2,4,1.0,...,3204.0,40.285714,31.466667,27.650000,28.866667,15.173599,23.116994,20.152659,20.596280,19.851381


In [3]:
merge_df_scaled.drop(columns=["item_id","dept_id","store_id","state_id","weekday"],inplace=True)

In [4]:
merge_df_scaled.fillna(0,inplace=True)

In [5]:
merge_df_scaled

,id,sales,wday,month,year,snap_CA,snap_TX,snap_WI,sell_price,cat_id_FOODS,...,rolling_sum_120,rolling_mean_7,rolling_mean_30,rolling_mean_60,rolling_mean_90,rolling_stdv_7,rolling_stdv_30,rolling_stdv_60,rolling_stdv_90,rolling_stdv_120
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,FOODS_2_197_CA_1_validation,38,1,1,0.0,0,0,0,0.598394,1.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-01-29,FOODS_3_080_CA_1_validation,33,1,1,0.0,0,0,0,0.297189,1.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-01-29,FOODS_3_090_CA_1_validation,107,1,1,0.0,0,0,0,0.251004,1.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-01-29,FOODS_3_120_CA_1_validation,0,1,1,0.0,0,0,0,0.000000,1.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-01-29,FOODS_3_252_CA_1_validation,19,1,1,0.0,0,0,0,0.297189,1.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-24,FOODS_3_555_CA_1_validation,24,2,4,1.0,0,0,0,0.337349,1.0,...,3169.0,33.571429,30.633333,27.083333,28.555556,19.789367,23.180675,19.909364,20.442911,19.716263
2016-04-24,FOODS_3_586_CA_1_validation,54,2,4,1.0,0,0,0,0.337349,1.0,...,3194.0,41.285714,30.933333,27.483333,28.800000,14.279522,23.429915,20.207812,20.615365,19.875319
2016-04-24,FOODS_3_587_CA_1_validation,26,2,4,1.0,0,0,0,0.497992,1.0,...,3204.0,40.285714,31.466667,27.650000,28.866667,15.173599,23.116994,20.152659,20.596280,19.851381


In [6]:
merge_df_scaled["id"].unique()

array(['FOODS_2_197_CA_1_validation', 'FOODS_3_080_CA_1_validation',
       'FOODS_3_090_CA_1_validation', 'FOODS_3_120_CA_1_validation',
       'FOODS_3_252_CA_1_validation', 'FOODS_3_555_CA_1_validation',
       'FOODS_3_586_CA_1_validation', 'FOODS_3_587_CA_1_validation',
       'FOODS_3_714_CA_1_validation', 'FOODS_3_808_CA_1_validation'],
      dtype=object)

# 1. Defining Model Functions

In [7]:
def perform_prophet(product_data):

    product_data.reset_index(inplace=True,names="date")
    
    prophet_product_df = product_data[["id","date","sales"]]
    prophet_product_df.columns = ["id","ds","y"]
    prophet_product_df['ds'] = pd.to_datetime(prophet_product_df['ds'])
    
    data_train = prophet_product_df.iloc[:-28]
    data_test = prophet_product_df.iloc[-28:]
    X_train = data_train["ds"]
    y_train = data_train["y"]
    X_test = data_test["ds"]
    y_test = data_test["y"]
    
    fbp = Prophet()

    model = fbp.fit(data_train)
    
    predict_placeholder = fbp.make_future_dataframe(28,freq="D")
    
    # Predict on the test data
    y_pred = fbp.predict(predict_placeholder[-28:])
    

    # Calculate and return the error metric for the current fold
    mae = mean_absolute_error(y_test, y_pred["yhat"])
    
    return model, mae

In [8]:
def perform_auto_arima(product_data):
    data_train = product_data.iloc[:-28]
    data_test = product_data.iloc[-28:]
    y_train = data_train["sales"]
    y_test = data_test["sales"]

    # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
    model = auto_arima(y_train, start_p=0, start_q=0, max_p=5, max_q=5, d=1,
                       seasonal=True, trace=False, error_action='ignore', 
                       suppress_warnings=True, stepwise=True)
    
    # Predict on the test data
    predictions = model.predict(n_periods=len(y_test))

    # Calculate and return the error metric for the current fold
    mae = mean_absolute_error(y_test, predictions)
    
    return model, mae

In [9]:
def objective_optuna(trial, y_train, y_test):
    
    trend = trial.suggest_categorical('trend', ['add'])
    seasonal = trial.suggest_categorical('seasonal', [None, 'add'])
    seasonal_periods = trial.suggest_categorical('seasonal_periods', [None, 4, 7, 12])
    
    product_results = []

    # Fit Holt-Winters model on the training data
    model = ExponentialSmoothing(y_train, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods,freq='D')
    fitted_model = model.fit(optimized=True)

    # Predict on the test data
    predictions = fitted_model.forecast(steps=len(y_test))

    # Calculate and store the error metric
    mae = mean_absolute_error(y_test, predictions)
    product_results.append(mae)

    # Average MAE for this product
    average_mae = np.mean(product_results)
    return average_mae

In [10]:
def perform_exp_smoothing(product_data):
    data_train = product_data.iloc[:-28]
    data_test = product_data.iloc[-28:]
    y_train = data_train["sales"]
    y_test = data_test["sales"]
    # Create a study object
    study = optuna.create_study(direction='minimize')
    
    print(f"Optimizing hyperparameters for product: {id}")
    
    
    # Run the optimization process for the current product
    study.optimize(lambda trial: objective_optuna(trial, y_train, y_test), n_trials=10, n_jobs=-1)

    # Get the best hyperparameters and the corresponding best MAE
    best_params = study.best_params
    best_mae = study.best_value

    # Create the best model with the obtained hyperparameters
    best_model = ExponentialSmoothing(y_train, **best_params).fit()
    
    return best_model, best_mae

In [11]:
import lightgbm as lgb

def perform_lightgbm(product_data):
    data_train = product_data.iloc[:-128]
    data_val = product_data.iloc[-128:-28]
    data_test = product_data.iloc[-28:]
    X_train = data_train.drop(columns="sales")
    y_train = data_train["sales"]
    X_val = data_val.drop(columns="sales")
    y_val = data_val["sales"]
    X_test = data_test.drop(columns="sales")
    y_test = data_test["sales"]

    
    
    
    # Define LightGBM parameters
    params = {
        "n_estimators": 1000,
        'objective': 'regression',
        'metric': 'rmse',
        "boosting_type": "gbdt",
        "max_depth": -1,
        'learning_rate': 0.01,
        'feature_fraction': 0.4,
        "lambda_l1": 1,
        "lambda_l2": 1,
        "seed": 46,
    }

    model = lgb.LGBMRegressor(**params)
    
    # Create dataset for LightGBM
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    
    # Train the model
    num_round = 1000

    bst = lgb.train(params, lgb_train, num_round, valid_sets=lgb_eval, callbacks=[lgb.early_stopping(stopping_rounds=50)])
     
    # Make predictions for the next 28 days
    predictions = bst.predict(X_test, num_iteration=bst.best_iteration)

    # Calculate and return the error metric for the current fold
    mae = mean_absolute_error(y_test, predictions)
    
    return bst, mae

# Example usage:
# sales_forecast = forecast_sales(product_data)
# print(sales_forecast)


# 2.Running all models in a loop to find for each product with lowest score

In [12]:
models_list = ["ARIMA","Prophet","ExponentialSmoothing","LightGBM"]

In [13]:
from pmdarima import auto_arima

# Dictionary to store MAE results for each unique time-series identified by id
product_results = {}
average_mae = []

# Iterate over each unique product series identified by id
for id in merge_df_scaled['id'].unique()[:10]:
    print(f"Analyzing product: {id}")
    product_data = merge_df_scaled[merge_df_scaled['id'] == id].drop(columns="id")
    product_data_with_id = merge_df_scaled[merge_df_scaled['id'] == id]

    # Results list for the current product time-series
    results = {}
    best_score = 999.99
    best_model_name = ""



    #Looping all models
    for model_name in models_list:

        if model_name == "ARIMA":
            #TODO: Add 5-fold split here for another loop (or inside the model function?) and then take the average score per model as their mae score
            
            # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
            model, mae = perform_auto_arima(product_data)
            results[model_name] = {"mae": mae, "model": model}
            if mae < best_score:
                best_score = mae
                best_model = model
                best_model_name = model_name

        elif model_name == "ExponentialSmoothing":

            # To be built
            model, mae = perform_exp_smoothing(product_data)
            results[model_name] = {"mae": mae, "model": model}
            if mae < best_score:
                best_score = mae
                best_model = model
                best_model_name = model_name

        elif model_name == "Prophet":

            model, mae = perform_prophet(product_data_with_id)
            results[model_name] = {"mae": mae, "model": model}
            if mae < best_score:
                best_score = mae
                best_model = model
                best_model_name = model_name


        elif model_name == "LightGBM":

            model, mae = perform_lightgbm(product_data)
            results[model_name] = {"mae": mae, "model": model}
            if mae < best_score:
                best_score = mae
                best_model = model
                best_model_name = model_name


    #Printing results for this product
    print(results)
    print(f"Model results for {id}")
    print(f"Best model: {best_model_name}")
    print(f"Best score: {best_score}")

    average_mae.append(best_score)

    # Store the average MAE for the current product time-series
    product_results[id] = {"best_score": best_score, "best_model": best_model_name, "model": best_model}

    #Store the best model in a pkl file
    filename = f'../models/{id}_model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(best_model, f)

# Create a DataFrame to store the results
results_df_arima = pd.DataFrame(product_results.items(), columns=['id', 'MAE'])

# Set the 'id' column as the index
results_df_arima.set_index('id', inplace=True)

average_mae = np.mean(average_mae)

print(f"Total average MAE: {average_mae}")


Analyzing product: FOODS_2_197_CA_1_validation


/tmp/ipykernel_76124/1072332254.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prophet_product_df['ds'] = pd.to_datetime(prophet_product_df['ds'])
11:52:55 - cmdstanpy - INFO - Chain [1] start processing
11:52:55 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-05-10 11:52:55,850] A new study created in memory with name: no-name-0805c0f4-be63-4fc2-8df0-3ae627c9e329


Optimizing hyperparameters for product: FOODS_2_197_CA_1_validation


[I 2024-05-10 11:52:57,431] Trial 3 finished with value: 9.783006525295232 and parameters: {'trend': 'add', 'seasonal': 'add', 'seasonal_periods': 4}. Best is trial 3 with value: 9.783006525295232.
[I 2024-05-10 11:52:57,499] Trial 2 finished with value: 9.388827437788668 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 7}. Best is trial 2 with value: 9.388827437788668.
[I 2024-05-10 11:52:57,648] Trial 7 finished with value: 9.388827437788668 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 12}. Best is trial 2 with value: 9.388827437788668.
[I 2024-05-10 11:52:57,684] Trial 9 finished with value: 9.388827437788668 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 4}. Best is trial 2 with value: 9.388827437788668.
[I 2024-05-10 11:52:57,693] Trial 4 finished with value: 9.388827437788668 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': None}. Best is trial 2 with value: 9.388827437788668.
[I 2024-05

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002606 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4593
[LightGBM] [Info] Number of data points in the train set: 1785, number of used features: 44
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 21.613445
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[413]	valid_0's rmse: 5.61169
{'ARIMA': {'mae': 8.726633436944372, 'model': ARIMA(order=(2, 1, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)}, 'Prophet': {'mae': 11.367010469983766, 'model': <prophet.forecaster.Prophet object at 0x7f1e17218ca0>}, 'ExponentialSmoothing': {'mae': 9.388827437788668, 'model': <statsmodels.tsa.holtwinters.results.HoltWintersResultsWrapper objec

...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003166 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4941
[LightGBM] [Info] Number of data points in the train set: 1785, number of used features: 44
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 28.964146
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[764]	valid_0's rmse: 6.87842
{'ARIMA': {'mae': 5.518778222112147, 'model': ARIMA(order=(5, 1, 5), scoring_args={}, suppress_warnings=True,
      with_intercept=False)}, 'Prophet': {'mae': 5.523726278892037, 'model': <prophet.forecaster.Prophet object at 0x7f1e16e92e30>}, 'ExponentialSmoothing': {'mae': 5.173390945001352, 'model': <stat

/tmp/ipykernel_76124/1072332254.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prophet_product_df['ds'] = pd.to_datetime(prophet_product_df['ds'])
12:00:38 - cmdstanpy - INFO - Chain [1] start processing
12:00:38 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-05-10 12:00:38,504] A new study created in memory with name: no-name-c1d7f3df-ea2e-4c2c-8406-185b3ab5c27b


Optimizing hyperparameters for product: FOODS_3_808_CA_1_validation


[I 2024-05-10 12:00:38,875] Trial 1 finished with value: 3.5378540430373917 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': None}. Best is trial 1 with value: 3.5378540430373917.
[I 2024-05-10 12:00:39,346] Trial 3 finished with value: 3.5378540430373917 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 7}. Best is trial 1 with value: 3.5378540430373917.
[I 2024-05-10 12:00:39,771] Trial 8 finished with value: 3.5378540430373917 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 4}. Best is trial 1 with value: 3.5378540430373917.
[I 2024-05-10 12:00:40,993] Trial 4 finished with value: 3.5201275671506607 and parameters: {'trend': 'add', 'seasonal': 'add', 'seasonal_periods': 4}. Best is trial 4 with value: 3.5201275671506607.
[I 2024-05-10 12:00:41,012] Trial 5 finished with value: 3.5201275671506607 and parameters: {'trend': 'add', 'seasonal': 'add', 'seasonal_periods': 4}. Best is trial 4 with value: 3.5201275671506607.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003513 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4946
[LightGBM] [Info] Number of data points in the train set: 1785, number of used features: 44
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 25.614006
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[993]	valid_0's rmse: 6.15928
{'ARIMA': {'mae': 3.219758128147768, 'model': ARIMA(order=(5, 1, 5), scoring_args={}, suppress_warnings=True,
      with_intercept=False)}, 'Prophet': {'mae': 10.665453491107941, 'model': <prophet.forecaster.Prophet object at 0x7f1e1b4fb2b0>}, 'ExponentialSmoothing': {'mae': 3.4805887917240503,

# ----------- Jonas Cleaned and Optimized until here -------------------

In [14]:
# Convert the results dictionary to a DataFrame
results_df_exp = pd.DataFrame(results_dict).T.reset_index()
results_df_exp.columns = ['id', 'ExpSmoothing_params', 'ExpSmoothing_MAE']

# Set the 'Product ID' column as the index
results_df_exp.set_index('id', inplace=True)
results_df_exp

NameError: name 'results_dict' is not defined

In [ ]:
# Merge the two DataFrames based on the product ID
comparison_df = pd.merge(results_df_arima, results_df_exp, left_index=True, right_index=True, suffixes=('_arima', '_exp'))
comparison_df['Best MAE'] = comparison_df[['ARIMA_MAE', 'ExpSmoothing_MAE']].min(axis=1)
comparison_df['Best Method'] = comparison_df.apply(lambda row: 'ARIMA' if row['Best MAE'] == row['ARIMA_MAE'] else 'Exponential Smoothing', axis=1)

comparison_df
